In [159]:
try:
    if is_init:
        pass
except:
    %pwd
    %cd ..
    %pwd
    is_init = True

In [160]:
from pathlib import Path
from src.utils import FileUtils
from config import Config
import torch
import numpy as np
import random
import itertools
data_folder = '../local_data'
work_folder = '../local_work'

### Unexplored Params

In [189]:
inc_data = torch.load(
                Path(f"{data_folder}/inconel-revised-raw-shuffled.pt"))["laser_params"]
params = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))["laser_params"] #stainless-steel-revised-shuffled inconel-revised-raw-shuffled
max_speed, max_spacing = params.max(0)[0][0].item(), params.max(0)[0][1].item()
min_speed, min_spacing = params.min(0)[0][0].item(), params.min(0)[0][1].item()
print(max_speed, max_spacing)
print(min_speed, min_spacing)

700.0 42.0
10.0 1.0


In [190]:
def denormalize_decode_result(y_hat, max_speed, max_spacing, min_speed, min_spacing):
    """input: 1,14 tensor
        output: 1,3 tensor with wattage no longer one hot encoded"""
    watts = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]

    watt_arg = torch.argmax(y_hat[0][2:])
    y_final = torch.empty(3, dtype=torch.float32)
    y_final[0] = max_speed * y_hat[0][0] # TODO call the scale
    y_final[1] = max_spacing * y_hat[0][1] # TODO call the scale
    y_final[2]= watts[watt_arg]
    return y_final

In [199]:
#identify explored param space
def get_unexplored_params(params, data, max_speed, max_spacing, min_speed, min_spacing, n = 196, seed = 1):
        """
        finds unexplored parameters
        inputs: tensor of [[], [], []], each size (1,14)
        outputs: list of n unnormalized tuples each of size (1,3)
        """
        param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
        #get set of all possible parameter combinations
        param_all_comb = set(list(itertools.product(*param_space)))#[list(tup) for tup in itertools.product(*param_space)]
        wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
        #get set of all training parameter combinations -decoding of one hot encoding
        train_param_space = set([tuple(x[:2] + [wattages[np.argmax(x[2:])]]) for x in data.tolist()]) #29988
        #find the combination parameters NOT in the training space
        unused_params = list(param_all_comb.symmetric_difference(train_param_space)) #len() = 5292
        #sample n from unused parameter combinations
        random.seed = seed
        indices = random.sample(range(len(unused_params)), n)
        return [unused_params[i] for i in indices]

def unexplored_params_to_tensors(x):
        new_x = []
        wattage_idxs = {
                0.2: 0,
                0.3: 1,
                0.4: 2,
                0.5: 3,
                0.6: 4,
                0.7: 5,
                0.8: 6,
                0.9: 7,
                1.0: 8,
                1.1: 9,
                1.2: 10,
                1.3: 11,
        }      
        for i in x:
                i = list(i)
                holder = i[:2] + np.zeros(12).tolist()
                holder[wattage_idxs[i[2]] + 2] = 1.0
                new_x.append(holder)

        laser_params = torch.FloatTensor(new_x)
        return laser_params / laser_params.max(0).values

#[torch.cat((x[:2], torch.tensor(np.array([wattages[np.argmax(x[2:])]])))) for x in params]#[:5]
#[x[:2] for x in params]#[:5]

# train_param_space = denormalize_decode_result(params, max_speed, max_spacing, min_speed, min_spacing)
# train_param_space


In [248]:
ouput_unexplored = get_unexplored_params(params, inc_data, max_speed, max_spacing, min_speed, min_spacing, n = 1000, seed = 1)
norm_unexplored_laser_params = unexplored_params_to_tensors(ouput_unexplored)

# Inverse Model Only


In [241]:
from src.models import InverseModel

In [242]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt


In [243]:
i_model = InverseModel.load_from_checkpoint(filepath, strict=False)
i_model.eval()

x = torch.from_numpy(np.array(torch.randn(1, 800) * 3)) #
with torch.no_grad():
    y_hat = i_model(x)
y_hat.shape

/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/spencersong/anaconda3/envs/meta/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:217: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['direct_model.model.0.weight', 'direct_model.model.0.bias', 'direct_model.model.2.layers.0.block.0.weight', 'direct_model.model.2.layers.0.block.0.bias', 'direct_model.model.2.layers.1.block.0.weight', 'direct_model.model.2.layers.1.block.0.bias', 'direct_model.model.2.layers.2.block.0.weight', 'direct_model.model.2.layers.2.block.0.bias', 'direct_model.model.2.layers.3.block.0.weight', 'direct_model.model.2.layers.3.block.0.bias', 'direct_model.model.2.layers.4.block.0.weight', 'direct_model.model.2.

torch.Size([1, 14])

In [204]:
# y_final2 = onehot_to_continuous2(y_hat)
print(denormalize_decode_result(y_hat,max_speed, max_spacing, min_speed, min_spacing))

#if you want hardcoded max, min values instead
#print(denormalize_decode_result(y_hat,700.0, 42.0, 10.0, 1.0))

tensor([ 1.9096, 39.3355,  1.1000])


# Direct Model Only


In [249]:
from src.models import DirectModel

In [250]:
filepath = Path(
    f"{work_folder}/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt")
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_D-ann-inconel-2022-07-05_14-39.ckpt


In [251]:
d_model = DirectModel.load_from_checkpoint(filepath)
d_model.eval()
# x = torch.randn(1, 14)
direct_y_hats = []
# intersect_preds = []
for i in norm_unexplored_laser_params:
    with torch.no_grad():
        direct_y_hats.append(d_model(i.reshape(1,14)))

### Unexplored Direct --> Indirect

In [252]:
filepath = Path(
    f"{work_folder}/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt") #CHANGEME inconel vs stainless best model
print(filepath)
if not Path.is_file(filepath):
    raise Exception(f'Model file does not exist at {filepath}!')

../local_work/saved_best/best_I-ann-inconel-2022-07-05_14-49.ckpt


In [253]:
model = InverseModel.load_from_checkpoint(filepath, strict=False)
model.eval()

#x = torch.from_numpy(np.array(torch.ra|ndn(1, 800) * 3)) #
param_preds = []
for i in direct_y_hats:
    with torch.no_grad():
        param_preds.append(model(i))#model(i))


In [254]:
decoded_preds = [denormalize_decode_result(pred, max_speed, max_spacing, min_speed, min_spacing) for pred in param_preds]
for r in decoded_preds:
    r[0]=torch.round(r[0], decimals = -1)
    r[1]=torch.round(r[1], decimals = 0)
decoded_preds

[tensor([670.0000,  13.0000,   1.2000]),
 tensor([320.0000,  19.0000,   1.2000]),
 tensor([630.0000,  12.0000,   1.2000]),
 tensor([590.0000,  16.0000,   1.2000]),
 tensor([410.0000,  40.0000,   0.8000]),
 tensor([520.0000,  18.0000,   1.2000]),
 tensor([620.0000,  14.0000,   1.2000]),
 tensor([400.0000,  37.0000,   0.8000]),
 tensor([60.0000, 41.0000,  0.8000]),
 tensor([6.7000e+02, 1.5000e+01, 5.0000e-01]),
 tensor([440.0000,  28.0000,   1.2000]),
 tensor([640.0000,  15.0000,   1.2000]),
 tensor([620.0000,  34.0000,   0.8000]),
 tensor([600.0000,  30.0000,   0.8000]),
 tensor([560.0000,  35.0000,   0.8000]),
 tensor([ 0.0000, 42.0000,  0.8000]),
 tensor([500.0000,  38.0000,   0.8000]),
 tensor([620.0000,  16.0000,   1.2000]),
 tensor([590.0000,  20.0000,   1.2000]),
 tensor([650.0000,  16.0000,   1.2000]),
 tensor([440.0000,  34.0000,   0.8000]),
 tensor([200.0000,  37.0000,   0.8000]),
 tensor([340.0000,  35.0000,   0.8000]),
 tensor([10.0000, 42.0000,  0.8000]),
 tensor([650.0000, 

In [257]:
param_space = [np.arange(min_speed, max_speed + 10, 10).tolist(), np.arange(min_spacing, max_spacing + 1, 1).tolist(), np.around(np.arange(.2, 1.4, .1), decimals = 1).tolist()]
#get set of all possible parameter combinations
param_all_comb = set(list(itertools.product(*param_space)))
wattages = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3]
space_params = set([tuple(np.around(x.tolist(), decimals= 1)) for x in decoded_preds]) #29988
        #find the combination parameters NOT in the training space
overlaps = list(param_all_comb.intersection(space_params)) #len() = 5292
#convert preds to set
len(overlaps)



618

In [258]:
overlaps[:195].tocsv()

[(170.0, 38.0, 0.8),
 (570.0, 13.0, 1.2),
 (590.0, 34.0, 0.8),
 (360.0, 40.0, 0.8),
 (270.0, 13.0, 1.2),
 (460.0, 16.0, 1.2),
 (270.0, 24.0, 1.2),
 (230.0, 22.0, 1.2),
 (610.0, 14.0, 1.2),
 (620.0, 28.0, 1.2),
 (180.0, 40.0, 0.8),
 (330.0, 35.0, 0.8),
 (390.0, 14.0, 1.2),
 (410.0, 35.0, 0.8),
 (320.0, 19.0, 1.2),
 (280.0, 17.0, 1.2),
 (270.0, 37.0, 0.8),
 (670.0, 12.0, 1.2),
 (160.0, 22.0, 1.2),
 (190.0, 38.0, 0.8),
 (370.0, 12.0, 1.2),
 (150.0, 36.0, 0.8),
 (330.0, 21.0, 1.2),
 (540.0, 31.0, 0.8),
 (500.0, 40.0, 0.8),
 (650.0, 10.0, 1.2),
 (310.0, 39.0, 0.8),
 (80.0, 31.0, 1.2),
 (680.0, 13.0, 0.5),
 (490.0, 13.0, 1.2),
 (560.0, 37.0, 0.8),
 (90.0, 38.0, 0.8),
 (430.0, 39.0, 0.8),
 (240.0, 33.0, 0.8),
 (610.0, 13.0, 1.2),
 (260.0, 21.0, 1.2),
 (130.0, 39.0, 0.8),
 (550.0, 35.0, 0.8),
 (520.0, 34.0, 0.8),
 (530.0, 14.0, 1.2),
 (70.0, 39.0, 0.8),
 (660.0, 17.0, 1.2),
 (290.0, 37.0, 0.8),
 (400.0, 19.0, 1.2),
 (90.0, 35.0, 1.2),
 (510.0, 12.0, 1.2),
 (640.0, 31.0, 0.5),
 (410.0, 38.0, 0.

In [256]:
#intersecting parameters!!!!!!!
intersect_x = []
wattage_idxs = {
        0.2: 0,
        0.3: 1,
        0.4: 2,
        0.5: 3,
        0.6: 4,
        0.7: 5,
        0.8: 6,
        0.9: 7,
        1.0: 8,
        1.1: 9,
        1.2: 10,
        1.3: 11,
}      
for i in unused_params:
        i = list(i)
        holder = i[:2] + np.zeros(12).tolist()
        holder[wattage_idxs[i[2]] + 2] = 1.0
        intersect_x.append(holder)
output = torch.tensor(intersect_x)
output

tensor([[600.,  34.,   0.,  ...,   0.,   0.,   0.],
        [590.,  14.,   0.,  ...,   0.,   1.,   0.],
        [580.,  31.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [640.,  16.,   0.,  ...,   0.,   1.,   0.],
        [520.,  38.,   0.,  ...,   0.,   0.,   0.],
        [300.,  37.,   0.,  ...,   0.,   0.,   0.]], dtype=torch.float64)

In [136]:
for i in range(len(params)):
    curr = params[i].flatten().tolist()
    if curr in output.tolist():
        print(i)

12058
19562
19609


In [149]:
from utils import rmse
# params[12058]
data = torch.load(
                Path(f"{data_folder}/stainless-steel-revised-shuffled.pt"))

intersect_train_emiss = []
intersect_train_emiss.append(data["emissivity"][12058])
intersect_train_emiss.append(data["emissivity"][19562])
intersect_train_emiss.append(data["emissivity"][19609])

# ONNX

In [ ]:
# import onnxruntime 
# from pathlib import Path
# import numpy as np
# import os

In [ ]:
ort_session = onnxruntime.InferenceSession(filepath)

In [ ]:
# filepath = Path(f"{work_folder}/saved_best/best_inference_direct.onnx")

# ort_session = onnxruntime.InferenceSession(filepath)
# input_name = ort_session.get_inputs()[0].name
# ort_inputs = {input_name: np.random.randn(2, 14)}
# ort_outs = ort_session.run(None, ort_inputs)
